<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/notebooks/10_heritage_by_address.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What kind of heritage is here?

For this script, we'll try to find out of what the heritage status of an address in Flanders is. Given a certain address, what do we know about the legal heritage status of it? Is it `beschermd` or not? Is it `vastgesteld` or not? We'll only take into account forms of legal protection. If it's been added to the inventory, but has not yet been `vastgesteld`, it won't appear here.

To make this possible, we need to use 2 different webservices:
* The [Digitaal Vlaanderen Location Service](https://loc.geopunt.be/Help/V4) that allows us to fetch information on an address in Flanders. It can tell us if the address exists (if not, possibly suggesting other addresses) and where the address is located geographically. This data and information is not being maintained by Onroerend Erfgoed.
* The [Onroerend Erfgoed Geozoekdiensten Service](https://geo.onroerenderfgoed.be/zoekdiensten/api_doc) that allows us to do a spatial query for heritage in Flanders, using a geometry (eg. a point or polygon).

For our purpose, we'll use the `location service` to turn an address into a point. Generally this will be the center of the administrative parcel the address is situated on or the building the address represents. Using this service is a two step process in itself. First, we ask is for suggestions given an address, then we take the first suggestion received to fetch the location of it. If there are multiple suggestions, we might want to change our input.

Once we have the location data, we'll then use the `geozoekdiensten service` with this point to do a spatial intersection with all Onroerend Erfgoed GIS layers. We can configure this service slightly by telling it to only check for `aanduidingsobjecten` (designated heritage) and to add a buffer of X meters to the point we're querying with. We'll also tell the service to not return the geometry of any heritage objects we find sinc this makes the reply heavier and we don't currently need that information.

In [ ]:
# Import required libraries
import requests
import json
import IPython

import logging

logging.basicConfig()

# Constants
LOC_HOST = 'https://loc.geopunt.be'
URL_LOC_SUGGESTION = LOC_HOST + '/v4/Suggestion'
URL_LOC = LOC_HOST + '/v4/Location'

GEO_HOST = 'https://geo.onroerenderfgoed.be'
URL_GEO_ZOEK = GEO_HOST + '/zoekdiensten/afbakeningen'

### Helper Functions ###

def get_adres_data(adres, session=None):
    '''
    Find information about an address through the Digital Vlaanderen geolocation service.

    Returns the requested address or False if it could not be found
    '''

    # If no session was passed, start a new one
    if not session:
        session = requests.Session()

    headers = {'Accept': 'application/json'}

    params = {'q': adres}

    # Get suggested adresses
    res = session.get(URL_LOC_SUGGESTION, headers=headers, params=params)
    res.raise_for_status()

    data = res.json()

    # The required SuggestionResult attribute was not present
    if not 'SuggestionResult' in data:
        return False

    # There are no SuggestionResults
    if not len(data['SuggestionResult']):
        return False

    # There are several SuggestionsResults, so issue a warning
    if len(data['SuggestionResult']) > 1:
        logging.warning(
            f"Multiple matching addresses found."
            f"Proceeding with the first of: {data['SuggestionResult']}"
        )

    # Work with the first suggestion and ask for more details
    adres = data['SuggestionResult'][0]

    params = {'q': adres, 'type': 'Housenumber'}

    res = session.get(URL_LOC, headers=headers, params=params)
    res.raise_for_status()

    data = res.json()

    # The required LocationResult attribute was not present
    if not 'LocationResult' in data:
        return False

    # The LocationResult had no data
    if not len(data['LocationResult']):
        return False

    return data['LocationResult'][0]

  
def find_objecten_op_adres(adres, buffer=5, session=None):
    '''
    Find objects located at a certain address.
    '''
    # If no session was passed, start a new one
    if not session:
        session = requests.Session()

    # Get the address data for this address
    adres_data = get_adres_data(adres, session)
    logging.debug('Adres data:', adres_data)

    # No address data was found, so we can't return any objects
    if not adres_data:
        logging.warning(f"Unable to locate Address {adres}")
        return []

    # Fetch x and y coordinates from the address data
    # in Lambert 72 projecteion
    x, y = adres_data['Location']['X_Lambert72'], adres_data['Location']['Y_Lambert72']

    # Construct a geometry to search with
    geom = {
        "type": "Point",
        "coordinates": [x, y],
        "crs": {
            "type": "name",
            "properties": {
                "name": "urn:ogc:def:crs:EPSG::31370"
            }
        }
    }

    headers = {'Accept': 'application/json'}

    params = {
        'categorie': 'aanduidingsobjecten',
        'geef_geometrie': 0,
        'buffer': buffer,
        'geometrie': json.dumps(geom)
    }

    response = session.get(URL_GEO_ZOEK, headers=headers, params=params)
    response.raise_for_status()
    return response.json()


# Our search address
# Should be written as <straat> <huisnummer>, <gemeente>
ADRES = 'Prinsstraat 13, Antwerpen'

#Initialize a session
session = requests.Session()

#Get all aanduidingsobjecten with buffer m of our address
aanduidingsobjecten = find_objecten_op_adres(ADRES, buffer=5, session=session)

#Generate markdown output
output = f"#Aanduidingsobjecten op {ADRES}\n"

for aanduidingsobject in aanduidingsobjecten:
    # Print URI and name of the Aanduidingsobjectobject
    output += f"* {aanduidingsobject['naam']} ({aanduidingsobject['uri']})\n"
    output += f"\t* *type:* {aanduidingsobject['type']['label']}\n"

IPython.display.Markdown(output)

# Conclusion

By combining two services, we have a powerful tool that allows us to see what kind of legal consequences attached to heritage are present in Flanders. It's possible we want more information on the objects we've found. We might want to know what type of heritage they are or how they were listed. Applying the lessons from [our first notebook](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/01_get_resource.ipynb), we simply do another GET request on the object's URI and we have all the information we could need.